## 中文分词

### 基于字标注的中文分词

字标注：
B | 词首
M | 词中
E | 词尾
O | 单字

我爱北京天安门

我/爱/北京/天安门

我/O 爱/O 北/B 京/E 天/B 安/M 门/E

## 命名实体识别

### 数据集及其介绍

命名实体识别就是把不属于实体的字用O标注，把实体用BME规则标注，最后按照BME规则把实体提取出来。

数据可以自己标注，也可以找个公开的数据集先练练手。为了方便，这里使用公开的数据集“玻森数据”提供的“命名实体识别数据”（https://bosonnlp.com/dev/resource ）

此数据集中共有6种实体类别：

例如：

### 数据预处理

1.首先，把原始数据按照BMEO规则变成字标注的形式，以便模型训练。

 定义origin2tag()函数，把原始数据按字标注，结果如下

2.其次，定义tagsplit()函数，按照标点符号把一个长句分成几个短句，并去除标点符号，结果如下

3.然后，建立一个word2id词典，按出现频次把每个汉字转换成id，id从1开始

再建立一个tag2id词典，把每一个字标注的类型转换成id（这里的id和上面的id不一样）

4.定义X_padding()函数把一句话用向量表示，如：

In [62]:
X_padding(['浙','江','在','线','杭','州','4','月','2','5','日','讯'])

[531,
 193,
 6,
 313,
 522,
 156,
 32,
 14,
 5,
 21,
 12,
 184,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

定义y_padding()函数把一句话的标签用向量表示，如：

In [61]:
y_padding(['B_product_name','M_product_name','M_product_name',
           'M_product_name','M_product_name','E_product_name',
           'B_time','M_time','M_time','M_time','E_time','O'],
)

[17,
 0,
 0,
 0,
 0,
 11,
 8,
 4,
 4,
 4,
 18,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

5.把每句话的字、字所对应的标签、x（句子对应字的向量表示）、y（句子对应标签的向量表示）存入df_data中，然后将x和y转为numpy数组

In [70]:
x,y

(array([[531, 193,   6, ...,   0,   0,   0],
        [132,  79,   4, ...,   0,   0,   0],
        [ 81, 315, 205, ...,   0,   0,   0],
        ...,
        [375, 373, 660, ...,   0,   0,   0],
        [ 66, 181,  19, ...,   0,   0,   0],
        [373, 660, 226, ...,   0,   0,   0]]),
 array([[17,  0,  0, ...,  0,  0,  0],
        [ 5,  5, 19, ...,  0,  0,  0],
        [ 5, 17, 11, ...,  0,  0,  0],
        ...,
        [ 5, 17,  0, ...,  0,  0,  0],
        [ 5,  5,  5, ...,  0,  0,  0],
        [17,  0, 11, ...,  0,  0,  0]]))

6.使用sklearn将x，y分为train和test集（test集比例为20%），把train集再分为train和valid集（valid集的比例为20%）。然后保存在Bosondata.pkl文件中（文件中还有word2id、id2word、tag2id、id2tag词典）

###### 代码：

In [1]:
import codecs
import pandas as pd
import numpy as np
import re
import collections
import sklearn

In [2]:
datas = list() # 字列表
labels = list() # 字所对应的标签列表
linedata = list() # 每句话的字列表
linelabel = list() # 每句话的字所对应的标签列表
tags = set() # 标签词典（所有出现过的标签）

In [3]:
input_data = codecs.open('./boson/wordtagsplit.txt', 'r', 'utf-8')
for line in input_data.readlines():
    line = line.split()
    linedata = []
    linelabel = []
    numNotO = 0
    for word in line:
        word = word.split('/')
        linedata.append(word[0])
        linelabel.append(word[1])
        tags.add(word[1])
        if word[1] != 'O':
            numNotO += 1
    if numNotO != 0:
        datas.append(linedata)
        labels.append(linelabel)
input_data.close()

In [4]:
datas

[['浙', '江', '在', '线', '杭', '州', '4', '月', '2', '5', '日', '讯'],
 ['记', '者', '', '施', '宇', '翔', '', '通', '讯', '员', '', '方', '英'],
 ['用', '微', '信', '交', '易', '毒', '品'],
 ['记', '者', '从', '杭', '州', '江', '干', '区', '公', '安', '分', '局', '了', '解', '到'],
 ['江',
  '干',
  '区',
  '禁',
  '毒',
  '专',
  '案',
  '组',
  '抓',
  '获',
  '吸',
  '贩',
  '毒',
  '人',
  '员',
  '5',
  '名'],
 ['黑', '龙', '江', '籍', '男', '子', '钱', '某', '长', '期', '落', '脚', '于', '宾', '馆'],
 ['经',
  '常',
  '半',
  '夜',
  '驾',
  '车',
  '来',
  '往',
  '于',
  '杭',
  '州',
  '主',
  '城',
  '区',
  '的',
  '各',
  '大',
  '宾',
  '馆',
  '和',
  '单',
  '身',
  '公',
  '寓'],
 ['并', '且', '常', '要', '活', '动', '到', '凌', '晨', '6', '、', '7', '点', '钟'],
 ['白', '天', '则', '在', '家', '里', '呼', '呼', '大', '睡'],
 ['钱', '某', '不', '寻', '常', '的', '特', '征'],
 ['发',
  '现',
  '钱',
  '某',
  '实',
  '际',
  '上',
  '是',
  '在',
  '向',
  '落',
  '脚',
  '于',
  '宾',
  '馆',
  '和',
  '单',
  '身',
  '公',
  '寓',
  '的',
  '吸',
  '毒',
  '人',
  '员',
  '贩',
  '送'],
 ['在',
  '即',
  '将',
  '举',
  

In [5]:
labels

[['B_product_name',
  'M_product_name',
  'M_product_name',
  'M_product_name',
  'M_product_name',
  'E_product_name',
  'B_time',
  'M_time',
  'M_time',
  'M_time',
  'E_time',
  'O'],
 ['O',
  'O',
  'B_person_name',
  'M_person_name',
  'M_person_name',
  'E_person_name',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B_person_name',
  'E_person_name'],
 ['O', 'B_product_name', 'E_product_name', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'B_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'E_org_name',
  'O',
  'O',
  'O'],
 ['B_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'M_org_name',
  'E_org_name',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_location',
  'M_location',
  'E_location',
  'O',
  'O',
  'O',
  'B_person_name',
  'E_person_name',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',

In [6]:
tags

{'',
 'B_company_name',
 'B_location',
 'B_org_name',
 'B_person_name',
 'B_product_name',
 'B_time',
 'E_company_name',
 'E_location',
 'E_org_name',
 'E_person_name',
 'E_product_name',
 'E_time',
 'M_company_name',
 'M_location',
 'M_org_name',
 'M_person_name',
 'M_product_name',
 'M_time',
 'O'}

In [7]:
def flatten(x): # 将x变为不可迭代对象
    result = []
    for el in x:
        if isinstance(x, collections.Iterable) and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

In [8]:
all_words = flatten(datas)
sr_allwords = pd.Series(all_words)  # 每个汉字出现的排序
sr_allwords = sr_allwords.value_counts()  # 每个字及其出现的频次
set_words = sr_allwords.index # 每个字的索引
set_ids = range(1, len(set_words)+1) # 每个字的序号，从1开始

In [9]:
tags = [i for i in tags]
tag_ids = range(len(tags)) # 标签的id
word2id = pd.Series(set_ids, index=set_words) # word2id词典，即可以根据汉字对应id
id2word = pd.Series(set_words, index=set_ids) # id2word词典，即可以根据id对应汉字
tag2id = pd.Series(tag_ids, index=tags) # tag2id词典，即可以根据标签对应id
id2tag = pd.Series(tags, index=tag_ids) # id2tag词典，即可以根据id对应tag

In [10]:
word2id['的']

1

In [11]:
id2word[1]

'的'

In [12]:
tag2id

B_location         0
                   1
E_location         2
E_company_name     3
M_person_name      4
M_company_name     5
E_org_name         6
M_time             7
B_company_name     8
B_product_name     9
B_org_name        10
M_product_name    11
O                 12
E_person_name     13
B_person_name     14
M_org_name        15
M_location        16
E_time            17
B_time            18
E_product_name    19
dtype: int64

In [13]:
word2id["unknow"] = len(word2id)+1
max_len = 60 # 设置一句话的最大长度

In [14]:
def X_padding(words): # 将一句话用向量表示
    ids = list(word2id[words])
    if len(ids) >= max_len:
        return ids[:max_len]
    ids.extend([0]*(max_len-len(ids)))
    return ids

In [15]:
def y_padding(tags): # 将一句话的标签用向量表示
    ids = list(tag2id[tags])
    if len(ids) >= max_len:
         return ids[:max_len]
    ids.extend([0]*(max_len-len(ids)))
    return ids

In [16]:
# 把每句话的字、字所对应的标签、x（句子对应字的向量表示）、y（句子对应标签的向量表示）
# 存入df_data中
df_data = pd.DataFrame(
        {'words': datas, 'tags': labels}, index=range(len(datas)))
df_data['x'] = df_data['words'].apply(X_padding)
df_data['y'] = df_data['tags'].apply(y_padding)
x = np.asarray(list(df_data['x'].values))
y = np.asarray(list(df_data['y'].values))

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=43)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train,  test_size=0.2, random_state=43)

In [21]:
x_train

array([[  4,   4,  51, ...,   0,   0,   0],
       [137,  51,  68, ...,   0,   0,   0],
       [809,  69, 153, ...,   0,   0,   0],
       ...,
       [ 51, 118,  24, ...,   0,   0,   0],
       [  5,  32, 107, ...,   0,   0,   0],
       [132,  80,  29, ...,   0,   0,   0]])

### 训练（BiLSTM+CRF）

#### BiLSTM   https://www.jiqizhixin.com/articles/2018-10-24-13

##### LSTM介绍

![jupyter](./pic/1.png)

![jupyter](./pic/2.png)

![jupyter](./pic/3.png)

![jupyter](./pic/4.png)

![jupyter](./pic/5.png)

##### BiLSTM介绍

![jupyter](./pic/6.png)

#### 条件随机场（CRF）  https://www.jianshu.com/p/55755fc649b1

   假设你有许多小明同学一天内不同时段的照片，从小明提裤子起床到脱裤子睡觉各个时间段都有（小明是照片控！）。现在的任务是对这些照片进行分类。比如有的照片是吃饭，那就给它打上吃饭的标签；有的照片是跑步时拍的，那就打上跑步的标签；有的照片是开会时拍的，那就打上开会的标签。问题来了，你准备怎么干？
    
   一个简单直观的办法就是，不管这些照片之间的时间顺序，想办法训练出一个多元分类器。就是用一些打好标签的照片作为训练数据，训练出一个模型，直接根据照片的特征来分类。例如，如果照片是早上6:00拍的，且画面是黑暗的，那就给它打上睡觉的标签;如果照片上有车，那就给它打上开车的标签。
    
   这样可行吗？乍一看可以！但实际上，由于我们忽略了这些照片之间的时间顺序这一重要信息，我们的分类器会有缺陷的。举个例子，假如有一张小明闭着嘴的照片，怎么分类？显然难以直接判断，需要参考闭嘴之前的照片，如果之前的照片显示小明在吃饭，那这个闭嘴的照片很可能是小明在咀嚼食物准备下咽，可以给它打上吃饭的标签；如果之前的照片显示小明在唱歌，那这个闭嘴的照片很可能是小明唱歌瞬间的抓拍，可以给它打上唱歌的标签。
    
   所以，为了让我们的分类器能够有更好的表现，在为一张照片分类时，我们必须将与它相邻的照片的标签信息考虑进来。这——就是条件随机场(CRF)大显身手的地方！

    从例子说起——词性标注问题
    
啥是词性标注问题？

非常简单的，就是给一个句子中的每个单词注明词性。比如这句话：“Bob drank coffee at Starbucks”，注明每个单词的词性后是这样的：“Bob (名词)  drank(动词)   coffee(名词)   at(介词)    Starbucks(名词)”。

下面，就用条件随机场来解决这个问题。

以上面的话为例，有5个单词，我们将：(名词，动词，名词，介词，名词)作为一个标注序列，称为l，可选的标注序列有很多种，比如l还可以是这样：（名词，动词，动词，介词，名词），我们要在这么多的可选标注序列中，挑选出一个最靠谱的作为我们对这句话的标注。

怎么判断一个标注序列靠谱不靠谱呢？

就我们上面展示的两个标注序列来说，第二个显然不如第一个靠谱，因为它把第二、第三个单词都标注成了动词，动词后面接动词，这在一个句子中通常是说不通的。

假如我们给每一个标注序列打分，打分越高代表这个标注序列越靠谱，我们至少可以说，凡是标注中出现了动词后面还是动词的标注序列，要给它负分！！

上面所说的动词后面还是动词就是一个特征函数，我们可以定义一个特征函数集合，用这个特征函数集合来为一个标注序列打分，并据此选出最靠谱的标注序列。也就是说，每一个特征函数都可以用来为一个标注序列评分，把集合中所有特征函数对同一个标注序列的评分综合起来，就是这个标注序列最终的评分值。

    定义CRF中的特征函数
    
现在，我们正式地定义一下什么是CRF中的特征函数，所谓特征函数，就是这样的函数，它接受四个参数：


· 句子s（就是我们要标注词性的句子）

· i，用来表示句子s中第i个单词

· l_i，表示要评分的标注序列给第i个单词标注的词性

· l_i-1，表示要评分的标注序列给第i-1个单词标注的词性


它的输出值是0或者1,0表示要评分的标注序列不符合这个特征，1表示要评分的标注序列符合这个特征。

Note:这里，我们的特征函数仅仅依靠当前单词的标签和它前面的单词的标签对标注序列进行评判，这样建立的CRF也叫作线性链CRF，这是CRF中的一种简单情况。为简单起见，本文中我们仅考虑线性链CRF。

